In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

The aim of this work is to involve as lot as possible of people in a space to talk about modelling and predictions with  the kaggle competitions. I think that exchange of ideas can improve us in our process of learning and growth as data scientist so feel free to comment if you find some insights in this notebook.
In the first part i'll explain  who i proceeded to got a score of almost 80% that could be used as a baseline.
You can use my notebook as inspiration and in the same time suggest me some modification.
One 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Reading the data

In [ ]:
train=pd.read_csv("/kaggle/input/titanic/train.csv")
train

In [ ]:
test=pd.read_csv("/kaggle/input/titanic/test.csv")
test

EDA

In [ ]:
#let's have information about datasets
train.info()
test.info()

In [ ]:
#Distribution of null values for training data
print(train.isnull().sum())
train.isnull().mean()

In [ ]:
#Distribution of null values for test data
print(test.isnull().sum())
test.isnull().mean()

In [ ]:
train.nunique()#@unique values for training data

In [ ]:
test.nunique()#unique values for testing data

Analizing Categorical data

Age

In [ ]:
train.Sex.value_counts()

In [ ]:
train.groupby("Sex")["Survived"].mean()#it seems gender is going to be  very interesting for our predictions

In [ ]:
pd.crosstab(train["Sex"],train["Survived"])

Pclass

In [ ]:
train.Pclass.value_counts()# the distribution for classes of trip

In [ ]:
train.groupby("Pclass")["Survived"].mean()#people in 1st class had more chances to survive

In [ ]:
pd.crosstab(train["Pclass"],train["Survived"])

SibSp

In [ ]:
train.SibSp.value_counts()#most of people were alone or withouth siblings or spouses

In [ ]:
train.groupby("SibSp")["Survived"].mean()#People with 1 or 2 siblings had more chances to survive

In [ ]:
pd.crosstab(train["SibSp"],train["Survived"])

Parch

In [ ]:
train.Parch.value_counts()

In [ ]:
pd.crosstab(train["Parch"],train["Survived"])#Similar to Sibsp. We have the maximum of chance to survive for a low value of Parch

In [ ]:
pd.crosstab(train["Parch"],train["Survived"])

Embarked

In [ ]:

train.Embarked.value_counts()

In [ ]:
pd.crosstab(train["Embarked"],train["Survived"])#Values for Embarked in Cherbourg seems better. Let's look at the correlation with Pclass

In [ ]:
pd.crosstab(train["Embarked"],train["Pclass"])#As we can see the most of Embarked in Cherbourg were in 1st class. They are not equally distributed for this variable

In [ ]:
train.groupby("Embarked")["Survived"].mean()

Quantitative Variable

In [ ]:
train.describe()#Some variable are more skewed, like Fare

In [ ]:
test.describe()

In [ ]:
train.corr()# Pclass(negatively) and Fare(positively) seems the more correlated with Age. Sibsp and Parch are correlated as well

In [ ]:
import seaborn as sns
plt.figure(figsize=(8,6))
sns.heatmap(train.corr())# we can see it better with the heatmap

In [ ]:
sns.pairplot(train)#let's take a look at the distributions. Many variables looks like categorical even they are numeric
plt.show()

Creation of new Variables

In [ ]:
train_b=train.copy()
train_b

Log Fare
We prefer to use the logarithm of Fare because of the high skewness

In [ ]:
train["log_Fare"]=np.log1p(train["Fare"])
train.head()

In [ ]:
test["log_Fare"]=np.log1p(test["Fare"])
test.head()

In [ ]:
train.skew()#As we can see the skewness is reduced with the new variable

Relatives.

We count the number of people nearest to the passenger ( Sibsp + Parch)

Not Alone.

A Variable who distinguuishes who is alone from others

In [ ]:
data = [train, test]
for dataset in data:
    dataset['relatives'] = dataset['SibSp'] + dataset['Parch']
    dataset.loc[dataset['relatives'] > 0, 'not_alone'] = 0
    dataset.loc[dataset['relatives'] == 0, 'not_alone'] = 1
    dataset['not_alone'] = dataset['not_alone'].astype(int)
train['not_alone'].value_counts()

In [ ]:
test["not_alone"].value_counts()

Deck

Distinguishes who has the Deck from others

In [ ]:
train.Cabin=train.Cabin.fillna("Missing")#We operate on Cabin variable to deal with missings. 
train.Cabin

In [ ]:
test.Cabin=test.Cabin.fillna("Missing")
test.Cabin

In [ ]:
#i create a function for finding the first letter of the cabin
def desk (string):
    prima=string[0]
    return prima

In [ ]:
train["deck"]=train["Cabin"].apply(desk)
train

In [ ]:
test["deck"]=test["Cabin"].apply(desk)
test

In [ ]:
pd.crosstab(train["deck"],train["Survived"])#We can clearly see that who doesn't have a deck had less chances to survive. We can modify Deck in a binary variable 

In [ ]:
train.groupby("deck")["Survived"].mean()

Ponte

Binary variable who distinguish who was withouth deck

In [ ]:
def ponte (string):
    if string=="M":
        return 0
    else:
        return 1
    
train["Ponte"]=train.deck.apply(ponte)
train.Ponte

In [ ]:
test["Ponte"]=test.deck.apply(ponte)
test.Ponte

In [ ]:
pd.crosstab(train.Ponte,train.Survived)#We can see the influence of this variable

In [ ]:
train.info()#We have now more variable ... we have to cut some of them to make a good choice for the model("Occam's Razor"). Except for Deck they are all numeric

In [ ]:
train.corr#Finally we can see that the new variables are correlated with "Survived" we can take them and delete other original variables

Before to proceed... we have still to manage some missing values

In [ ]:
train.isnull().mean()# We will impute the median for  Age because is more robust to outliers.. and we will impute the mode for Embarked

In [ ]:
test.isnull().mean()# like below we will impute the median for age and for the few missings for log_Fare( Fare doesn't interest us we will exclude it)

In [ ]:
train["Age"]=train["Age"].fillna(train["Age"].median())#Imputation for Age in the train set
train.isnull().mean()

In [ ]:
train["Embarked"]=train["Embarked"].fillna("S") # Southampton is the mode for the distribution 
train.isnull().mean()

In [ ]:
test["Age"]=test["Age"].fillna(test["Age"].median()) # imputation for the median of Age in test set 
test.isnull().mean()

In [ ]:
test["log_Fare"]=test["log_Fare"].fillna(test["log_Fare"].median()) # Imputation for log_Fare in test set
test.isnull().mean()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

y = train["Survived"]

features = ["Pclass", "Sex","log_Fare", "Age","Embarked","Ponte",'relatives','not_alone']
X = pd.get_dummies(train[features])
X_test = pd.get_dummies(test[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

In [ ]:
test